In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, svm
from sklearn.model_selection import train_test_split, cross_val_score
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.utils import compute_class_weight


from sklearn.metrics import accuracy_score, f1_score,confusion_matrix,recall_score,precision_score,log_loss,roc_auc_score,roc_curve

In [ ]:
X = pd.read_csv ('/content/x_rosmap_whole_gene_expression_downsampled.csv')

y = pd.read_csv ('/content/y_rosmap_whole_gene_expression_downsampled.csv')

In [ ]:
X.shape

(5355, 190)

In [ ]:
X = X.transpose()

In [ ]:
y = y.transpose()

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [ ]:
#logistic regression
log_reg = linear_model.LogisticRegression(
    penalty='l1',
    C=1.0,
    solver='liblinear')

log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import numpy as np
log_coef = log_reg.coef_

In [ ]:
pd.DataFrame(log_coef).to_csv('log_coef.csv')  

In [ ]:
print(accuracy_score(y_test,y_pred))

0.6842105263157895


In [ ]:
print(f1_score(y_test,y_pred))
print(roc_auc_score(y_test,y_pred))

0.6842105263157895
0.6861111111111111


In [ ]:
#random forest

In [161]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [162]:
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(max_depth=2, random_state=0)

In [163]:
rf_pred = clf.predict(X_test)

In [164]:
print(accuracy_score(y_test,rf_pred))

0.6578947368421053


In [ ]:
clf_coef = clf.feature_importances_

In [ ]:
pd.DataFrame(clf_coef).to_csv('clf_coef.csv')  

In [ ]:
print(f1_score(y_test,rf_pred))
print(roc_auc_score(y_test,rf_pred))

0.6666666666666666
0.6611111111111112


In [ ]:
n_methods = 4
n_fold = 5
skf = StratifiedKFold(n_splits=n_fold, random_state=40,shuffle=True) # stratified k fold partitiion shared by all methods
# parameters to tune for each classifier
rf_tuned_parameters = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy'],
    'random_state' : [40]
} 
svm_tuned_parameters =  [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]},
                    {'kernel': ['sigmoid'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000] },{'kernel': ['linear'], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}]     

In [ ]:
# empty array to save coefficient from elstic net and lasso models
el_coef = []
lasso_coef = []

# empty array to save model performance
# empty array to save model performance
elnet_accuracy = []
lasso_accuracy = []
SVM_accuracy = []
RF_accuracy = []
elnet_f1 = []
lasso_f1 = []
SVM_f1 = []
RF_f1 = []
elnet_roc_acu = []
lasso_roc_auc = []
SVM_roc_auc = []
RF_ruc_auc = []

## training performance
elnet_accuracy_tr = []
lasso_accuracy_tr = []
SVM_accuracy_tr = []
RF_accuracy_tr = []

elnet_f1_tr = []
lasso_f1_tr = []
SVM_f1_tr = []
RF_f1_tr = []

elnet_roc_acu_tr = []
lasso_roc_auc_tr = []
SVM_roc_auc_tr = []
RF_ruc_auc_tr = []

x_train=[]
x_test=[]
y_train=[]
y_test=[]
i=0
# nested 5-fold cross validation
for train_index, test_index in skf.split(x, y):

  tr_perf = list() # empty list for performance
  test_perf = list()

  X_train = x.loc[train_index,:]
  X_test = x.loc[test_index,:]
  Y_train = y.loc[train_index]
  Y_test = y.loc[test_index]

  x_train.append(X_train)
  x_test.append(X_test)
  y_train.append(Y_train)
  y_test.append(Y_test)
  print("data fold no.",i)
  i=i+1
  
  #Normalize data
  scaler = StandardScaler().fit(X_train)
  normalized_X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns = X_train.columns
    )
  normalized_X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns = X_test.columns
    )
  
  # Logistic regression CV with elastic net penalty
  elnet = LogisticRegressionCV(
    l1_ratios= [0.0001,0.001,0.01,0.1, 0.5, 0.7, 0.9, 0.95, 0.99],
    penalty='elasticnet',
    solver='saga',
    Cs=100,
    max_iter=100, ## change may need
    tol=0.01, ## change may need
    cv= skf
    )
  elnet.fit(X=normalized_X_train, y=Y_train)
  #el_coef.append(elnet.coef_)
  el_c=elnet.coef_
  el_coef.append(el_c)
  metric_train = elnet.score(X = normalized_X_train, y = Y_train) # scoring = ['accuracy', 'f1','roc_auc']
  metric_test = elnet.score(X = normalized_X_test, y = Y_test) #scoring = ['accuracy', 'f1','roc_auc']
  print("training performance")
  y_elnet_trpred=elnet.predict(normalized_X_train)
  
  y_elnet_pred = elnet.predict(normalized_X_test)
  print("test performance")
  print("elastic net Accuracy:",metrics.accuracy_score(Y_test, y_elnet_pred))
  print("elastic net f1:",metrics.f1_score(Y_test, y_elnet_pred))
  print("elastic net roc_auc_score:",roc_auc_score(Y_test, y_elnet_pred))
  tr_perf.append(metric_train)
  test_perf.append(metric_test)

  # Logistic regression CV with lasso penalty
  lasso = LogisticRegressionCV(
    penalty='l1',
    Cs=100,
    solver='liblinear', # added by bing
    max_iter=150,
    tol=0.01,
    cv= skf
    )
  lasso.fit(X=normalized_X_train, y=Y_train)
  #lasso_coef.append(lasso.coef_)
  lasso_c=lasso.coef_
  lasso_coef.append(lasso_c)
  metric_train = lasso.score(X = normalized_X_train, y = Y_train) # scoring = ['accuracy', 'f1','roc_auc']
  metric_test = lasso.score(X = normalized_X_test, y = Y_test) # scoring = ['accuracy', 'f1','roc_auc']
  print("training performance")
  y_lasso_trpred=lasso.predict(normalized_X_train)

  print("test performance")
  y_lasso_pred = lasso.predict(normalized_X_test)
  print("lasso Accuracy:",metrics.accuracy_score(Y_test, y_lasso_pred))
  print("lasso f1:",metrics.f1_score(Y_test, y_lasso_pred))
  print("lasso roc_auc_score:",roc_auc_score(Y_test, y_lasso_pred))
  tr_perf.append(metric_train)
  test_perf.append(metric_test)

  #SVM CV
  svc = svm.SVC()
  clf = GridSearchCV(svc, svm_tuned_parameters, cv=skf, scoring='accuracy').fit(normalized_X_train, Y_train)
  #optimized_clf.fit(normalized_X_train, Y_train)
  optimized_clf = clf.best_estimator_
  metric_train = optimized_clf.score(X = normalized_X_train, y = Y_train) #scoring = ['accuracy', 'f1','roc_auc']
  metric_test = optimized_clf.score(X = normalized_X_test, y = Y_test) #scoring = ['accuracy', 'f1','roc_auc']
  print("training performance")
  y_SVM_trpred=optimized_clf.predict(normalized_X_train)

  print("test performance")
  y_SVM_pred = optimized_clf.predict(normalized_X_test)
  print("SVM Accuracy:",metrics.accuracy_score(Y_test, y_SVM_pred))
  print("SVM f1:",metrics.f1_score(Y_test, y_SVM_pred))
  print("SVM roc_auc_score:",roc_auc_score(Y_test, y_SVM_pred))
  tr_perf.append(metric_train)
  test_perf.append(metric_test)

  #randome forest CV
  clf = GridSearchCV(RandomForestClassifier(), rf_tuned_parameters, cv=skf, scoring='accuracy').fit(normalized_X_train, Y_train) #['accuracy', 'f1','roc_auc']
  optimized_RF = clf.best_estimator_
  #optimized_clf.fit(normalized_X_train, Y_train)
  metric_train = optimized_RF.score(X = normalized_X_train, y = Y_train) #, scoring = ['accuracy', 'f1','roc_auc']
  metric_test = optimized_RF.score(X = normalized_X_test, y = Y_test) #scoring = ['accuracy', 'f1','roc_auc'])
  print("training performance")
  y_RF_trpred=optimized_RF.predict(normalized_X_train)

  print("test performance")
  y_RF_pred = optimized_RF.predict(normalized_X_test)
  print("randome forest Accuracy:",metrics.accuracy_score(Y_test, y_RF_pred))
  print("randome forest f1:",metrics.f1_score(Y_test, y_RF_pred))
  print("randome forest roc_auc_score:",roc_auc_score(Y_test, y_RF_pred))
  tr_perf.append(metric_train)
  test_perf.append(metric_test)
  
  # save 6 performance tables to excel file
  # initialize data of lists.

  ### testing performance
  el_acc=metrics.accuracy_score(Y_test, y_elnet_pred)
  elnet_accuracy.append(el_acc)
  lasso_acc=metrics.accuracy_score(Y_test, y_lasso_pred)
  lasso_accuracy.append(lasso_acc)
  svm_acc=metrics.accuracy_score(Y_test, y_SVM_pred)
  SVM_accuracy.append(svm_acc)
  rf_acc=metrics.accuracy_score(Y_test, y_RF_pred)
  RF_accuracy.append(rf_acc)

  el_f1=metrics.f1_score(Y_test, y_elnet_pred)
  elnet_f1.append(el_f1)
  las_f1=metrics.f1_score(Y_test, y_lasso_pred)
  lasso_f1.append(las_f1)
  svm_f1=metrics.f1_score(Y_test, y_SVM_pred)
  SVM_f1.append(svm_f1)
  rf_f1=metrics.f1_score(Y_test, y_RF_pred)
  RF_f1.append(rf_f1)
  el_roc=roc_auc_score(Y_test, y_elnet_pred)
  elnet_roc_acu.append(el_roc)
  lass_roc=roc_auc_score(Y_test, y_lasso_pred)
  lasso_roc_auc.append(lass_roc)
  svm_roc=roc_auc_score(Y_test, y_SVM_pred)
  SVM_roc_auc.append(svm_roc)
  rf_roc=roc_auc_score(Y_test, y_SVM_pred)
  RF_ruc_auc.append(rf_roc)

  ## training performance
  el_tr_acc=metrics.accuracy_score(Y_train, y_elnet_trpred)
  elnet_accuracy_tr.append(el_tr_acc)
  las_tr_acc=metrics.accuracy_score(Y_train, y_lasso_trpred)
  lasso_accuracy_tr.append(las_tr_acc)
  svm_tr_acc=metrics.accuracy_score(Y_train, y_SVM_trpred)
  SVM_accuracy_tr.append(svm_tr_acc)
  rf_tr_acc=metrics.accuracy_score(Y_train, y_RF_trpred)
  RF_accuracy_tr.append(rf_tr_acc)

  el_tr_f1=metrics.f1_score(Y_train, y_elnet_trpred)
  elnet_f1_tr.append(el_tr_f1)
  las_tr_f1=metrics.f1_score(Y_train, y_lasso_trpred)
  lasso_f1_tr.append(las_tr_f1)
  svm_tr_f1=metrics.f1_score(Y_train, y_SVM_trpred)
  SVM_f1_tr.append(svm_tr_f1)
  rf_tr_f1=metrics.f1_score(Y_train, y_RF_trpred)
  RF_f1_tr.append(rf_tr_f1)

  el_tr_roc=roc_auc_score(Y_train, y_elnet_trpred)
  elnet_roc_acu_tr.append(el_tr_roc)
  las_tr_roc=roc_auc_score(Y_train, y_lasso_trpred)
  lasso_roc_auc_tr.append(las_tr_roc)
  svm_tr_roc=roc_auc_score(Y_train, y_SVM_trpred)
  SVM_roc_auc_tr.append(svm_tr_roc)
  rf_tr_roc=roc_auc_score(Y_train, y_SVM_trpred)
  RF_ruc_auc_tr.append(rf_tr_roc)

  

In [ ]:
# save 5 fold data 
for i in range(5):
    a=x_train[i]
    b=y_train[i]
    c=x_test[i]
    d=y_test[i]
    a.to_csv('FREESURFER/thick_x_train'+str(i)+'.csv')
    b.to_csv('FREESURFER/thick_y_train'+str(i)+'.csv')
    c.to_csv('FREESURFER/thick_x_test' + str(i) + '.csv')
    d.to_csv('FREESURFER/thick_y_test' + str(i) + '.csv')

In [ ]:
# Save the coefficient of elastic net and lasso to files: el_coef, lasso_coef
for i in range(5):
    a=lasso_coef[i]
    a=pd.DataFrame(a)
    a.to_csv('FREESURFER/classifier_results/thick__lasso_coef_fold'+str(i)+'.csv')

for i in range(5):
    b=el_coef[i]
    b=pd.DataFrame(b)
    b.to_csv('FREESURFER/classifier_results/thick__el_coef_fold'+str(i)+'.csv')

In [ ]:
# empty array to save coefficient from elstic net and lasso models
el_coef = []
lasso_coef = []

# empty array to save model performance
# empty array to save model performance
elnet_accuracy = []
lasso_accuracy = []
SVM_accuracy = []
RF_accuracy = []
elnet_f1 = []
lasso_f1 = []
SVM_f1 = []
RF_f1 = []
elnet_roc_acu = []
lasso_roc_auc = []
SVM_roc_auc = []
RF_ruc_auc = []

## training performance
elnet_accuracy_tr = []
lasso_accuracy_tr = []
SVM_accuracy_tr = []
RF_accuracy_tr = []

elnet_f1_tr = []
lasso_f1_tr = []
SVM_f1_tr = []
RF_f1_tr = []

elnet_roc_acu_tr = []
lasso_roc_auc_tr = []
SVM_roc_auc_tr = []
RF_ruc_auc_tr = []

x_train=[]
x_test=[]
y_train=[]
y_test=[]
i=0
# nested 5-fold cross validation
for train_index, test_index in skf.split(x, y):

  tr_perf = list() # empty list for performance
  test_perf = list()

  X_train = x.loc[train_index,:]
  X_test = x.loc[test_index,:]
  Y_train = y.loc[train_index]
  Y_test = y.loc[test_index]

  x_train.append(X_train)
  x_test.append(X_test)
  y_train.append(Y_train)
  y_test.append(Y_test)
  print("data fold no.",i)
  i=i+1
  
  #Normalize data
  scaler = StandardScaler().fit(X_train)
  normalized_X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns = X_train.columns
    )
  normalized_X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns = X_test.columns
    )
  
  # Logistic regression CV with elastic net penalty
  elnet = LogisticRegressionCV(
    l1_ratios= [0.0001,0.001,0.01,0.1, 0.5, 0.7, 0.9, 0.95, 0.99],
    penalty='elasticnet',
    solver='saga',
    Cs=100,
    max_iter=100, ## change may need
    tol=0.01, ## change may need
    cv= skf
    )
  elnet.fit(X=normalized_X_train, y=Y_train)
  #el_coef.append(elnet.coef_)
  el_c=elnet.coef_
  el_coef.append(el_c)
  metric_train = elnet.score(X = normalized_X_train, y = Y_train) # scoring = ['accuracy', 'f1','roc_auc']
  metric_test = elnet.score(X = normalized_X_test, y = Y_test) #scoring = ['accuracy', 'f1','roc_auc']
  print("training performance")
  y_elnet_trpred=elnet.predict(normalized_X_train)
  
  y_elnet_pred = elnet.predict(normalized_X_test)
  print("test performance")
  print("elastic net Accuracy:",metrics.accuracy_score(Y_test, y_elnet_pred))
  print("elastic net f1:",metrics.f1_score(Y_test, y_elnet_pred))
  print("elastic net roc_auc_score:",roc_auc_score(Y_test, y_elnet_pred))
  tr_perf.append(metric_train)
  test_perf.append(metric_test)